In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import evaluate
import torch

def training_function():
    raw_datasets = load_dataset("glue", "mrpc")
    checkpoint = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def tokenize_function(example):
        return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    tokenized_datasets.set_format("torch")
    print("column names:", tokenized_datasets["train"].column_names)

    train_dataloader = DataLoader(
        tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
    )
    eval_dataloader = DataLoader(
        tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
    )

    accelerator = Accelerator()

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    optimizer = AdamW(model.parameters(), lr=3e-5)

    train_dl, eval_dl, model, optimizer = accelerator.prepare(
        train_dataloader, eval_dataloader, model, optimizer
    )

    num_epochs = 3
    num_training_steps = num_epochs * len(train_dl)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    progress_bar = tqdm(range(num_training_steps))

    model.train()
    for epoch in range(num_epochs):
        for batch in train_dl:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)


    metric = evaluate.load("glue", "mrpc")
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    print("metric:", metric.compute())

In [3]:
from accelerate import notebook_launcher
notebook_launcher(training_function, num_processes = 3)

Launching training on 3 GPUs.
column names: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
column names: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
column names: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yaojg/miniconda3/envs/d2l/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yaojg/miniconda3/envs/d2l/lib/python3.10/site-packages/transformers/optimizati

  0%|          | 0/459 [00:00<?, ?it/s]

  0%|          | 0/459 [00:00<?, ?it/s]

  0%|          | 0/459 [00:00<?, ?it/s]

metric: {'accuracy': 0.8602941176470589, 'f1': 0.9015544041450777}
metric: {'accuracy': 0.8602941176470589, 'f1': 0.9015544041450777}
metric: {'accuracy': 0.8602941176470589, 'f1': 0.9015544041450777}
